### **어텐션 가중치(attention weights)**
- 선형 결합을 통한, 전체 시퀀스의 가중 평균
    $$ x'_{i} = \sum_{j=1}^{n} w_{ji}x_{j}$$
    $ x'_{i} : \text{linear combination} $ (선형 결합)<Br>
    계수(coefficient) 는 $ w_{ji} : \text{attention weights} $ 총합이 1 -> **softmax 함수 이용**
    
    선형 결합으로 생성된 임베딩 : **문맥 고려 임베딩(contextualized embedding)**

### seq2seq 의 attention 매커니즘

![seq2seq with attention](https://github.com/tommyjin2894/KDT_project2/blob/main/0_2_3%20%EC%A1%B0%20%EB%AA%A8%EB%8D%B8%20%EA%B5%AC%EC%A1%B0%20%EC%84%A4%EB%AA%85%EB%8F%84_1.png?raw=true)

[사용 예시 프로젝트2 : 혐호 표현 분류 모델 (seq2seq with attention)](https://github.com/tommyjin2894/KDT_project2)

- 인코더
    ```py
    encoder_inputs = Input(shape=(max_text_len,))
    encoder_embedding = Embedding(corpus_size, embedding_size1, trainable=True, mask_zero=True)(encoder_inputs)
    encoder_dropout = Dropout(dropout_ratio)(encoder_embedding)
    encoder_lstm1 = Bidirectional(LSTM(lstm_size, return_state=True, return_sequences=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm1(encoder_dropout)
    ```

- 디코더
    ```py
    decoder_inputs = Input(shape=(max_summary_len,))
    decoder_embedding_layer = Embedding(corpus_size_answer, embedding_size2, trainable=True, mask_zero=True)
    decoder_embedding = decoder_embedding_layer(decoder_inputs)
    decoder_dropout = Dropout(dropout_ratio)(decoder_embedding)
    decoder_lstm1 = LSTM(lstm_size*2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm1(decoder_dropout, initial_state=encoder_states)
    decoder_outputs = Dropout(dropout_ratio)(decoder_outputs)
    ```

- 어텐션 레이어
    ```py
    attention_layer = Attention()
    attention_result = attention_layer([decoder_outputs, encoder_outputs])
    decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_result])
    ```

- 출력층
    ```py
    decoder_dense = Dense(corpus_size_answer, activation='softmax')
    decoder_outputs = decoder_dense(decoder_concat_input)
    ```
    
- 모델 정의
    ```py
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    ```

- 어텐션 레이어의 계산
   1. **어텐션 스코어 계산**:
      - $ \text{Attention Score} = h_t \cdot h_s^T $

   2. **소프트맥스 적용**:
      - 어텐션 스코어에 소프트맥스를 적용하여 가중치 $ \alpha_s $로 변환.
      - $ \alpha_s = \text{softmax}\left(\frac{h_t \cdot h_s^T}{\sqrt{d_h}}\right) $
      - 여기서 $ d_h $는 hidden state의 차원입니다.

   3. **출력 계산**:
      - 가중치 $ \alpha_s $를 인코더의 출력 $ h_s $에 곱하여 최종 출력을 계산.
      - $ \text{Output} = \sum_{s=1}^{S} \alpha_s \cdot h_s $
   4. 여기에서 인코더의 $h_s$ 는 $K$ 와 $V$ 역할, 디코더의 $h_t$ 는 $Q$의 역할

### 기존 seq2seq 및 + attention 이용의 단점
- 순차적 특성으로 인하여 병렬 처리 불가(병렬에 유리한 gpu기능의 제약적 활용)
    - 순차적인 학습으로 인하여 경사 소실 문제가 발생 문제.
- bidirectional lstm을 이용함에도 한계점이 있다. 또한 여전히 순차적인 계산 이용한다.
- attention 층의 과부하
    - 모든 층의 hs cs를 concat 하여 계산하기 때문에 시퀀스가 길어짐에 따라 과부하가 걸린다.

### [트랜스 포머](https://arxiv.org/pdf/1706.03762)
![transformers](https://i.sstatic.net/BpxYv.png)

|순서|인코더 부분|디코더 부분|
|---|---|---|
|0|입력 임베딩 + 포지셔널 인코딩|입력 임베딩 + 포지셔널 인코딩|
|1|셀프 어텐션 (멀티 헤드 어텐션) |마스크드 셀프 어텐션 (멀티 헤드 어텐션)|
|2|잔차 연결|잔차 연결|
|3|레이어 정규화|레이어 정규화|
|4|피드 포워드 네트워크|디코더-인코더 어텐션 (멀티 헤드 어텐션)|
|5|잔차 연결|잔차 연결|
|6|레이어 정규화|레이어 정규화|
|7|인코더의 출력|피드 포워드 네트워크|
|8||잔차 연결|
|9||레이어 정규화|
|10||선형 층|
|11||소프트맥스 출력|

- 논문에서의 멀티헤드 어텐션은 8개 


### 다양한 트랜스 포머 모델들
- **Encoder** + **Decoder** : t5
- **Encoder** 모델 : Bert 류 등
    * 양방향 어텐션(bidirectional attention)
- **Decoder** 모델 : gpt 류 등
    * 자기 회귀 어텐션(autoregressive attention)


### **셀프 어텐션**

![image.png](https://img.notionusercontent.com/s3/prod-files-secure%2F1f406a34-9833-41b3-af5a-06ebe0821494%2F21f6d610-d50d-4e43-b38d-27b010a1153a%2F1_self_attention.svg/size/?exp=1731833224&sig=-78EuHKYI03NfVZHf57uJkKR1m8ghm-5BOYRelCDV0U)

- **스케일드 닷 어텐션(점곱 어텐션)** 
    1. positional embedding 생성(위치정보)
    2. 쿼리(Q), 키(K), 밸류(V)의 $ W_Q , W_K ,  W_V $와 곱해져 프로젝션
        $$ Q = XW_Q, \quad K = XW_K, \quad V = XW_V $$
    
    3. **어텐션 점수** 계산 : **쿼리(Q)** 와 **키(K)의** **내적(dot product)**
        $$ \text{score}(i,j) = Q_i \cdot K_j^T $$
        $$\text{score\_scaled}(i, j) = \frac{Q_i \cdot K_j^T}{\sqrt{d_k}}$$
        - 안정성 추가 : $\sqrt{d_k}$ 
          - $d_k$:입력 벡터의 차원 수
    
    3. **어텐션 가중치(attention weights) 계산**:
        - 스케일된 점수를 소프트맥스 함수를 통해 변환하여 어텐션 가중치를 얻습니다:
          $$
          w_{ij} = \text{softmax}(\text{score\_scaled}(i, j)) \\
          = \text{softmax}(\frac{Q_i \cdot K_j^T}{\sqrt{d_k}})
          $$
    
    4. **토큰 임베딩 업데이트 (어텐션 출력)**:
        - 어텐션 가중치를 통해 밸류(V)의 가중 평균을 계산하여 새로운 출력 벡터 생성
          $$ x'_{i} = \sum_{j=1}^{n} w_{ij} v_{j} $$
          $$ \therefore x'_i \text{(output vector)} = \sum_{j=1}^{n} \text{softmax}\left(\frac{Q_i \cdot K_j^T}{\sqrt{d_k}}\right) V_j $$
          - $i$ = 쿼리 벡터의 인덱스 - 즉 각 쿼리마다의 어텐션 출력인 w'_i 이다.
          - $j$ = 키 벡터의 인덱스
          - $d_k$ = 키 벡터의 차원